# سیشن 5 – ملٹی ایجنٹ آرکسٹریٹر

Foundry Local کا استعمال کرتے ہوئے ایک سادہ دو ایجنٹ پائپ لائن (ریسرچر -> ایڈیٹر) کا مظاہرہ کرتا ہے۔


### وضاحت: ڈیپینڈنسی انسٹالیشن
`foundry-local-sdk` اور `openai` انسٹال کرتا ہے جو مقامی ماڈل تک رسائی اور چیٹ مکمل کرنے کے لیے ضروری ہیں۔ یہ عمل بار بار کرنے پر بھی کوئی مسئلہ پیدا نہیں کرتا۔


# منظر نامہ
ایک کم سے کم دو ایجنٹ آرکسٹریٹر پیٹرن نافذ کرتا ہے:
- **ریسرچر ایجنٹ** مختصر اور حقائق پر مبنی نکات جمع کرتا ہے
- **ایڈیٹر ایجنٹ** ان نکات کو ایگزیکٹو وضاحت کے لیے دوبارہ لکھتا ہے

مشترکہ میموری فی ایجنٹ، درمیانی نتائج کا ترتیب وار تبادلہ، اور ایک سادہ پائپ لائن فنکشن کا مظاہرہ کرتا ہے۔ مزید کرداروں (جیسے نقاد، تصدیق کنندہ) یا متوازی شاخوں کے لیے توسیع پذیر۔

**ماحولیاتی متغیرات:**
- `FOUNDRY_LOCAL_ALIAS` - استعمال کرنے کے لیے ڈیفالٹ ماڈل (ڈیفالٹ: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - بنیادی ایجنٹ ماڈل (ALIAS کو اووررائیڈ کرتا ہے)
- `AGENT_MODEL_EDITOR` - ایڈیٹر ایجنٹ ماڈل (بنیادی ماڈل پر ڈیفالٹ)

**SDK حوالہ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**یہ کیسے کام کرتا ہے:**
1. **FoundryLocalManager** خود بخود Foundry Local سروس شروع کرتا ہے
2. مخصوص ماڈل ڈاؤن لوڈ اور لوڈ کرتا ہے (یا کیش شدہ ورژن استعمال کرتا ہے)
3. تعامل کے لیے OpenAI کے موافق اینڈ پوائنٹ فراہم کرتا ہے
4. ہر ایجنٹ مختلف ماڈل استعمال کر سکتا ہے خصوصی کاموں کے لیے
5. بلٹ ان ریٹری لاجک عارضی ناکامیوں کو خوش اسلوبی سے ہینڈل کرتا ہے

**اہم خصوصیات:**
- ✅ خودکار سروس دریافت اور ابتدائیہ
- ✅ ماڈل لائف سائیکل مینجمنٹ (ڈاؤن لوڈ، کیش، لوڈ)
- ✅ OpenAI SDK مطابقت ایک مانوس API کے لیے
- ✅ ایجنٹ کی مہارت کے لیے ملٹی ماڈل سپورٹ
- ✅ مضبوط خرابی ہینڈلنگ ریٹری لاجک کے ساتھ
- ✅ مقامی انفرنس (کلاؤڈ API کی ضرورت نہیں)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### وضاحت: کور امپورٹس اور ٹائپنگ  
ایجنٹ پیغام ذخیرہ کرنے کے لیے ڈیٹاکلاسز اور وضاحت کے لیے ٹائپنگ ہنٹس متعارف کراتا ہے۔ بعد کے ایجنٹ کے اعمال کے لیے فاؤنڈری لوکل مینیجر اور اوپن اے آئی کلائنٹ کو امپورٹ کرتا ہے۔


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### وضاحت: ماڈل کی ابتدا (SDK پیٹرن)
Foundry Local Python SDK کا استعمال مضبوط ماڈل مینجمنٹ کے لیے:
- **FoundryLocalManager(alias)** - سروس کو خودکار طور پر شروع کرتا ہے اور ماڈل کو عرفیت کے ذریعے لوڈ کرتا ہے
- **get_model_info(alias)** - عرفیت کو ٹھوس ماڈل ID میں تبدیل کرتا ہے
- **manager.endpoint** - OpenAI کلائنٹ کے لیے سروس کا اینڈ پوائنٹ فراہم کرتا ہے
- **manager.api_key** - API کلید فراہم کرتا ہے (مقامی استعمال کے لیے اختیاری)
- مختلف ایجنٹس (پرائمری بمقابلہ ایڈیٹر) کے لیے الگ ماڈلز کی حمایت کرتا ہے
- مضبوطی کے لیے بلٹ ان ریٹری لاجک کے ساتھ ایکسپونینشل بیک آف
- سروس کے تیار ہونے کو یقینی بنانے کے لیے کنکشن کی تصدیق

**اہم SDK پیٹرن:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**لائف سائیکل مینجمنٹ:**
- مینیجرز کو عالمی طور پر ذخیرہ کیا جاتا ہے تاکہ مناسب صفائی ہو سکے
- ہر ایجنٹ ماہرین کے لیے مختلف ماڈل استعمال کر سکتا ہے
- خودکار سروس دریافت اور کنکشن ہینڈلنگ
- ناکامیوں پر ایکسپونینشل بیک آف کے ساتھ شائستہ ریٹری

یہ ایجنٹ آرکسٹریشن شروع ہونے سے پہلے مناسب ابتدا کو یقینی بناتا ہے۔

**حوالہ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### وضاحت: ایجنٹ اور میموری کلاسز
ہلکی پھلکی `AgentMsg` میموری اندراجات کے لیے اور `Agent` کو شامل کرتی ہے:
- **سسٹم کردار** - ایجنٹ کی شخصیت اور ہدایات
- **پیغام کی تاریخ** - گفتگو کے سیاق و سباق کو برقرار رکھتی ہے
- **act() طریقہ** - مناسب غلطی ہینڈلنگ کے ساتھ اعمال انجام دیتا ہے

ایجنٹ مختلف ماڈلز (پرائمری بمقابلہ ایڈیٹر) استعمال کر سکتا ہے اور ہر ایجنٹ کے لیے الگ تھلگ سیاق و سباق برقرار رکھتا ہے۔ یہ پیٹرن ممکن بناتا ہے:
- اعمال کے دوران میموری کا تسلسل
- ہر ایجنٹ کے لیے ماڈل کی لچکدار تفویض
- غلطیوں کی تنہائی اور بحالی
- آسان چیننگ اور آرکیسٹریشن


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### وضاحت: مربوط پائپ لائن  
دو خاص ایجنٹس تخلیق کرتا ہے:  
- **ریسرچر**: بنیادی ماڈل استعمال کرتا ہے، حقائق پر مبنی معلومات جمع کرتا ہے  
- **ایڈیٹر**: الگ ماڈل استعمال کر سکتا ہے (اگر ترتیب دیا گیا ہو)، معلومات کو بہتر بناتا اور دوبارہ لکھتا ہے  

`pipeline` فنکشن:  
1. ریسرچر خام معلومات جمع کرتا ہے  
2. ایڈیٹر اسے ایگزیکٹو کے لیے تیار آؤٹ پٹ میں بہتر بناتا ہے  
3. درمیانی اور حتمی نتائج دونوں واپس کرتا ہے  

یہ طریقہ کار فراہم کرتا ہے:  
- ماڈل کی تخصیص (مختلف کرداروں کے لیے مختلف ماڈلز)  
- معیار میں بہتری، کئی مراحل کی پروسیسنگ کے ذریعے  
- معلومات کی تبدیلی کی قابلِ سراغی  
- مزید ایجنٹس یا متوازی پروسیسنگ کے لیے آسان توسیع  


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### وضاحت: پائپ لائن کی عملدرآمد اور نتائج
ایک کمپلائنس + لیٹنسی سے متعلق سوال پر ملٹی ایجنٹ پائپ لائن کو چلانے کا مظاہرہ:
- معلومات کی ملٹی اسٹیج تبدیلی
- ایجنٹ کی مہارت اور تعاون
- نتائج کے معیار میں بہتری کے ذریعے اصلاح
- ٹریس ایبلٹی (دونوں درمیانی اور حتمی نتائج محفوظ)

**نتائج کی ساخت:**
- `question` - صارف کا اصل سوال
- `research` - خام تحقیق کا نتیجہ (حقائق پر مبنی نکات)
- `final` - بہتر کردہ ایگزیکٹو خلاصہ
- `models` - ہر مرحلے کے لیے استعمال کیے گئے ماڈلز

**توسیعی خیالات:**
1. معیار کے جائزے کے لیے ایک Critic ایجنٹ شامل کریں
2. مختلف پہلوؤں کے لیے متوازی تحقیقاتی ایجنٹس نافذ کریں
3. حقائق کی تصدیق کے لیے ایک Verifier ایجنٹ شامل کریں
4. مختلف پیچیدگی کی سطحوں کے لیے مختلف ماڈلز استعمال کریں
5. تکراری بہتری کے لیے فیڈبیک لوپس نافذ کریں


### ایڈوانسڈ: کسٹم ایجنٹ کنفیگریشن

ایجنٹ کے رویے کو حسب ضرورت بنانے کی کوشش کریں، ابتدائی سیل چلانے سے پہلے ماحول کے متغیرات میں ترمیم کریں:

**دستیاب ماڈلز:**
- تمام دستیاب ماڈلز دیکھنے کے لیے ٹرمینل میں `foundry model ls` استعمال کریں
- مثالیں: phi-4-mini، phi-3.5-mini، qwen2.5-7b، llama-3.2-3b، وغیرہ۔


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**ڈس کلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لیے کوشش کرتے ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا خامیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
